<a href="https://colab.research.google.com/github/JeonJeongMin/tensorflow-examples/blob/master/Supervised%20Learning/RNN02_wordComplete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import module

In [0]:
import tensorflow as tf
import numpy as np

char_arr = ['a','b','c','d','e','f','g',
            'h','i','j','k','l','m','n',
            'o','p','q','r','s','t','u',
            'v','w','x','y','z']
num_dic = {n: i for i, n in enumerate(char_arr)}
dic_len = len(num_dic)

#Make a word


In [0]:
seq_data = ['abcd','word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']
def make_batch(seq_data):
  input_batch=[]
  target_batch=[]
  
  for seq in seq_data:
    input = [num_dic[n] for n in seq[:-1]]
    target = num_dic[seq[-1]]
    input_batch.append(np.eye(dic_len)[input])#one_hot_encoding
    target_batch.append(target)
    
  return input_batch, target_batch


#Setting Parameter

In [0]:
learning_rate = 0.01
n_hidden =128
total_epoch = 50

n_step = 3
n_input = n_class = dic_len

#Make a Multi-RNN network

In [0]:
tf.reset_default_graph()
X = tf.placeholder(tf.float32, [None, n_step, n_input])
Y = tf.placeholder(tf.int32, [None])

cell1 = tf.nn.rnn_cell.LSTMCell(n_hidden)
cell1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob = 0.5)
cell2 = tf.nn.rnn_cell.LSTMCell(n_hidden)

multi_cell = tf.nn.rnn_cell.MultiRNNCell([cell1,cell2])

outputs, states = tf.nn.dynamic_rnn(multi_cell, X, dtype=tf.float32)

outputs = tf.transpose(outputs,[1,0,2])
outputs = outputs[-1]
model = tf.layers.dense(outputs, n_class, activation = None)

cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
                        logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

#Learning

In [9]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, target_batch = make_batch(seq_data)

print('Learning Start!')

for epoch in range(total_epoch):
  _, cost_val = sess.run([optimizer,cost], feed_dict={X:input_batch, Y:target_batch})
  print('Epoch:{:02d}'.format(epoch+1), 'Cost:%3f'%cost_val)
  
print('Learning Finish!')

Learning Start!
Epoch:01 Cost:3.264422
Epoch:02 Cost:3.126595
Epoch:03 Cost:2.892318
Epoch:04 Cost:2.328626
Epoch:05 Cost:1.563359
Epoch:06 Cost:1.695839
Epoch:07 Cost:1.341806
Epoch:08 Cost:1.002718
Epoch:09 Cost:1.083760
Epoch:10 Cost:0.993529
Epoch:11 Cost:0.795121
Epoch:12 Cost:0.641947
Epoch:13 Cost:0.540534
Epoch:14 Cost:0.514443
Epoch:15 Cost:0.409392
Epoch:16 Cost:0.351125
Epoch:17 Cost:0.332804
Epoch:18 Cost:0.262500
Epoch:19 Cost:0.369718
Epoch:20 Cost:0.292628
Epoch:21 Cost:0.311017
Epoch:22 Cost:0.215484
Epoch:23 Cost:0.187132
Epoch:24 Cost:0.231440
Epoch:25 Cost:0.211865
Epoch:26 Cost:0.158947
Epoch:27 Cost:0.154033
Epoch:28 Cost:0.260898
Epoch:29 Cost:0.252150
Epoch:30 Cost:0.103076
Epoch:31 Cost:0.121701
Epoch:32 Cost:0.110077
Epoch:33 Cost:0.141599
Epoch:34 Cost:0.126803
Epoch:35 Cost:0.108686
Epoch:36 Cost:0.175021
Epoch:37 Cost:0.106488
Epoch:38 Cost:0.210099
Epoch:39 Cost:0.133590
Epoch:40 Cost:0.085403
Epoch:41 Cost:0.097421
Epoch:42 Cost:0.041355
Epoch:43 Cost:0.12

#Check Accuracy

In [10]:
prediction = tf.cast(tf.argmax(model,1),tf.int32)
prediction_check = tf.equal(prediction,Y)
accuracy = tf.reduce_mean(tf.cast(prediction_check,tf.float32))

input_batch, target_batch = make_batch(seq_data)

predict, accuracy_val = sess.run([prediction, accuracy], feed_dict = {X:input_batch, Y:target_batch})

predict_word = []
for idx,val in enumerate(seq_data):
  last_char = char_arr[predict[idx]]
  predict_word.append(val[:3]+last_char)

print('입력값:', [w[:3]+' ' for w in seq_data])
print('예측값:', predict_word)
print('정확도:', accuracy_val)

입력값: ['abc ', 'wor ', 'woo ', 'dee ', 'div ', 'col ', 'coo ', 'loa ', 'lov ', 'kis ', 'kin ']
예측값: ['abcd', 'word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']
정확도: 1.0
